In [2]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
# import statsmodels.api as sm
import csv
# import pycircos
from scipy.stats import poisson
import itertools
import collections
import seaborn as sns
from jupyter_locuszoom import LocusZoom

import FACSus as fs

sns.set(style="whitegrid")

from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
%cd /gpfs3/well/ansari/users/gjx698/BOSON_FACS

[Errno 2] No such file or directory: '/gpfs3/well/ansari/users/gjx698/BOSON_FACS'
/gpfs3/well/ansari/users/gjx698/boson/scripts


In [ ]:
''